In [1]:
import requests
import pandas as pd
import key

## PV Systems in API data

The `pv_systems` data set initialy consists of 158 PV plants. For every record there are information about it name, id, and location (`site_id`, `site_location`, `site_latitude`, `site_longitude` and `site_elevation`). From all that site data the `system_id` will be used as request parameter for the site metadata.

In [2]:
pv_systems = pd.read_csv('systems.csv')

In [3]:
pv_systems.head()

,system_id,system_public_name,site_id,site_public_name,site_location,site_latitude,site_longitude,site_elevation
0,4902,NIST_Ground_1,3197,s_NIST_G1,Gaithersburg MD,39.1319,-77.2141,138.0
1,4903,NIST_Roof_1,3198,s_NIST_R1,Gaithersburg MD,39.1354,-77.2156,149.0
2,4901,NIST_Canopy_1,3196,s_NIST_C1,Gaithersburg MD,39.1385,-77.2155,137.0
3,2,Residential 1a,1,Residential 1,"Golden, CO",39.7214,-105.0972,1675.0
4,50,NREL x-Si 6,3,SERF East,"Golden, CO",39.7420,-105.1727,1994.7


In [4]:
pv_systems.shape

(158, 8)

There is no missing data for `system_id`, althought there are duplicates of sites. Those will be dropped in order to avoid making requests for the same PV plant twice. After deleting any duplicated sites there are total of 156 PV plants for which metadata will be extracted.

In [5]:
pv_systems['system_id'].isna().unique()

array([False])

In [6]:
(pv_systems['system_id'].value_counts() > 1).any()

True

In [7]:
pv_systems = pv_systems.drop_duplicates()

In [8]:
(pv_systems['system_id'].value_counts() > 1).any()

False

In [9]:
pv_systems.shape

(156, 8)

## Request metadata for every PV system

In [10]:
metadata_report = []
error_request = []

In [13]:
for system in pv_systems['system_id']:
    result_metadata = requests.get(f'https://developer.nrel.gov/api/pvdaq/v3/sites.json?api_key={key.key}&system_id={system}')
    if result_metadata.status_code == 200:
        metadata_report.append(result_metadata.json())
    elif result_metadata.status_code == 401:
        error_request.append({'error_type': '401-Unauthorized', 'system_id': system})
    elif result_metadata.status_code == 422:
        error_request.append({'error_type': '422-Unprocessable Entity', 'system_id': system})

## JSON report to CSV

In [14]:
meta_dataset = pd.DataFrame.from_dict(metadata_report[0]['outputs'])
for system_index in range(1, len(metadata_report)):
    meta_dataset = pd.concat([pd.DataFrame.from_dict(metadata_report[system_index]['outputs']), meta_dataset], ignore_index=True)
meta_dataset

,available_years,comments,confidential,inverter_mfg,inverter_model,module_mfg,module_model,site_power,module_tech,name_private,name_public,site_area,site_azimuth,site_elevation,site_latitude,site_longitude,site_tilt,system_id
0,"[2010, 2011, 2012, 2013, 2014, 2015, 2016, 201...",Having repeated problems with inverter main bo...,False,SatCon Technology,135kW,Sharp,NU-U240F1,146640.0,1,Andre Agassi Preparatory Academy - Building A,[34] Andre Agassi Preparatory Academy - Buildi...,996.0278,180.0,620.0,36.1952,-115.1582,11.2,34
1,"[2015, 2016, 2017]",None,False,SMA,Tripower 20kW,SolarWorld,SW 280 Mono,40000.0,None,"Clark County, NV - Hollywood Rec Center","[1419] Clark County, NV - Hollywood Rec Center",None,210.0,779.0,36.1534,115.0256,10.0,1419
2,[],None,False,Solectria,PVI 3000,Suntech,STP-270,3240.0,2,"Make It Right, New Orleans, Site #1",Residential - NOLA #1 - District A North,23.28,185.9,10.0,30.0017,90.0926,17.0,1244
3,"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",None,False,NaN,NaN,NaN,NaN,140140.0,1,Distributed Sun - 5 Executive Campus,[1201] Distributed Sun - 5 Executive Campus,959.058,230.0,18.0,39.9283,-75.0481,10.0,1201
4,"[2009, 2010, 2011, 2012, 2013, 2014, 2015, 201...",1-axis tracked,False,Advanced Energy,Solaron 333 kVA,Evergreen Solar,"ES-190, ES-195",1082850.0,None,"NREL, Windsite PV 1-axis",[1431] NREL Windsite PV 1-axis,None,None,1850.0,39.9094,105.2311,None,1431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,"[2010, 2011, 2012, 2013, 2014, 2016, 2017, 201...",None,False,Fronius,IG 3000,UniSolar,PVL-136,2720.0,3,"Nelson Home, Golden, CO, Array 2",[3] Residential #1b,43.23,181.2,1675.0,39.7214,-105.0972,18.5,3
237,"[2010, 2011, 2012, 2013, 2014, 2015, 2016, 201...",None,False,SMA,Sunny Boy 3000US,Silicor Materials,240,2400.0,9,"OTF, Calisolar (System 1)",[33] Silicor Materials,15.76,180.0,1794.5,39.7404,-105.1772,40.0,33
238,"[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]",Array tilt is a weighted average of the two ti...,False,Satcon,"PVS-500(x2), PVS-135(x1)",SunPower,"318W(x3136), 315W(496)",1153488.0,1,"NREL, Parking Garage",[1332] NREL Parking Garage,5922.7534,180.0,1770.0,39.7388,-105.1732,16.77,1332
239,"[1822, 1994, 1995, 1996, 1997, 1998, 1999, 200...",None,False,Fronius,IG 4500-LV,Siemens,M55 c-SI,6000.0,None,"SERF, Serf East Systems",[50] NREL x-Si #6,None,158.0,1994.7,39.7420,-105.1727,45.0,50


In [15]:
meta_dataset.to_csv('systems_metadata')